## Import Modules

In [ ]:
import tensorflow as tf
from keras.models import Sequential,load_model
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
import os
import fnmatch
import numpy as np
import boto3
import matplotlib.pyplot as plt
import cv2
import patoolib
%matplotlib inline

## Fetch the bucket in which the data is stored on S3

In [ ]:
s3 = boto3.resource('s3')

## Create Alexnet Model

In [ ]:
def create_model():
    #Create a sequential model
    model = Sequential()

    # 1st Convolutional Layer
    model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11),\
     strides=(4,4), padding='valid'))
    model.add(Activation('relu'))
    # Pooling 
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation before passing it to the next layer
    model.add(BatchNormalization())

    # 2nd Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 3rd Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 4th Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 5th Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # Passing it to a dense layer
    model.add(Flatten())
    # 1st Dense Layer
    model.add(Dense(4096, input_shape=(224*224*3,)))
    model.add(Activation('relu'))
    # Add Dropout to prevent overfitting
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 2nd Dense Layer
    model.add(Dense(4096))
    model.add(Activation('relu'))
    # Add Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 3rd Dense Layer
    model.add(Dense(1000))
    model.add(Activation('relu'))
    # Add Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # Output Layer
    model.add(Dense(1,activation='linear'))
    
    
    return model

## Build the Model

In [ ]:
model = create_model()
model.summary()


## Compile the Model

In [ ]:
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])
print("Model Compiled")

## Split the training and validation set

In [ ]:
def extract_x(folder):
    x = []
    folder = folder + '/'
    for root,dirs,files in os.walk(folder):
        for f in files:
            if fnmatch.fnmatch(f,"*.png"):
                img = cv2.imread(folder+f)
                img = cv2.resize(img,(224,224))
                x.append(img)
                #print(f)
    print(folder+"completed")
    lb = int(0.6*len(x))
    return np.array(x[:lb]),np.array(x[lb:])



def extract_y(folder):
    folder = folder+'/'
    y = np.load(folder+'angles1.npy')
    lb = int(0.6*len(y))
    return np.array(y[:lb]),np.array(y[lb:])

## Define Keras callbacks for accuracy improvement

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=20, verbose=0,
                                  mode='min')
#ckpt = ModelCheckpoint('.model.hdf5', save_best_only=True,monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                       patience=7, verbose=1, min_delta=1e-4,
                                       mode='min')

## Train the model

In [ ]:
prev = []
for object in s3.Bucket(name='kneeapp').objects.all():
    if len(prev) > 0:
        os.remove(prev[0])
        os.remove(prev[0][:-4])
        print("Removed"+prev[0])
    s3.Bucket('kneeapp').download_file(object.key, object.key)
    print("Downloaded"+object.key)
    patoolib.extract_archive(object.key, outdir="./")
    print("Extracted")
    prev.append(object.key)
    train_x,valid_x = extract_x(object.key[:-4])
    train_y,valid_y = extract_y(object.key[:-4])
    
    
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


    datagen.fit(train_x)

    print(train_x.shape)
    print(train_y.shape)

    # fits the model on batches with real-time data augmentation:
    history = model.fit_generator(datagen.flow(train_x, np.expand_dims(train_y,-1), batch_size=32),steps_per_epoch=len(train_x) / 32, epochs=3,verbose=1,callbacks=[earlyStopping, reduce_lr_loss], validation_data=(valid_x, valid_y))
    
    #history = model.fit(train_x, train_y, batch_size=64, epochs=3,verbose=1,callbacks=[earlyStopping, reduce_lr_loss],validation_data=(valid_x, valid_y))
    
    

### Visualize Training and Validation Loss

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()